In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/I1/bismillah_fiks.csv')
df

,Product ID,Qty,Posting Date Inbound,Posting Date Outbound
0,92248141,12,2022-05-11,2022-05-11
1,92248142,6,2022-05-11,2022-05-11
2,92248143,1,2022-05-11,2022-05-11
3,92248144,5,2022-05-11,2022-05-11
4,92248144,5,2022-05-11,2022-05-11
...,...,...,...,...
42892,92282037,1,2022-10-31,2022-10-31
42893,92282040,36,2022-10-31,2022-10-31
42894,92282041,12,2022-10-31,2022-10-31
42895,92282042,3,2022-10-31,2022-10-31


In [ ]:
df[["Posting Date Inbound", "Posting Date Outbound"]] = df[["Posting Date Inbound", "Posting Date Outbound"]].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42897 entries, 0 to 42896
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Product ID             42897 non-null  int64         
 1   Qty                    42897 non-null  int64         
 2   Posting Date Inbound   42897 non-null  datetime64[ns]
 3   Posting Date Outbound  42897 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 1.3 MB


In [ ]:
df.isnull().sum()

Product ID               0
Qty                      0
Posting Date Inbound     0
Posting Date Outbound    0
dtype: int64

In [ ]:
df['Range'] = df['Posting Date Outbound'] - df['Posting Date Inbound']
df

,Product ID,Qty,Posting Date Inbound,Posting Date Outbound,Range
0,92248141,12,2022-05-11,2022-05-11,0 days
1,92248142,6,2022-05-11,2022-05-11,0 days
2,92248143,1,2022-05-11,2022-05-11,0 days
3,92248144,5,2022-05-11,2022-05-11,0 days
4,92248144,5,2022-05-11,2022-05-11,0 days
...,...,...,...,...,...
42892,92282037,1,2022-10-31,2022-10-31,0 days
42893,92282040,36,2022-10-31,2022-10-31,0 days
42894,92282041,12,2022-10-31,2022-10-31,0 days
42895,92282042,3,2022-10-31,2022-10-31,0 days


In [ ]:
df = df.drop(['Posting Date Inbound', 'Posting Date Outbound'], axis=1)
df

,Product ID,Qty,Range
0,92248141,12,0 days
1,92248142,6,0 days
2,92248143,1,0 days
3,92248144,5,0 days
4,92248144,5,0 days
...,...,...,...
42892,92282037,1,0 days
42893,92282040,36,0 days
42894,92282041,12,0 days
42895,92282042,3,0 days


In [ ]:
df['Range'] = (df['Range'] / np.timedelta64(1, 'D')).astype('float64')
print(df['Range'])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
42892    0.0
42893    0.0
42894    0.0
42895    0.0
42896    0.0
Name: Range, Length: 42897, dtype: float64


In [ ]:
df['Range'].value_counts()

 1.0     19346
 0.0     11148
-1.0      6731
 2.0      3605
-2.0      1036
 3.0       739
-3.0       219
 33.0       62
 11.0        3
 9.0         2
 4.0         2
 7.0         2
 6.0         1
 14.0        1
Name: Range, dtype: int64

In [ ]:
df['Range'] = df['Range'].astype('int64')
df

,Product ID,Qty,Range
0,92248141,12,0
1,92248142,6,0
2,92248143,1,0
3,92248144,5,0
4,92248144,5,0
...,...,...,...
42892,92282037,1,0
42893,92282040,36,0
42894,92282041,12,0
42895,92282042,3,0


In [ ]:
features = df.drop('Range', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, df['Range'], random_state=None)

exported_pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(bootstrap=False, max_features=1.0, min_samples_leaf=4, min_samples_split=3, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

In [ ]:
print(exported_pipeline.score(testing_features, testing_target))

0.9843466621773416


In [ ]:
train_pred = exported_pipeline.predict(training_features)
train_mse = mean_squared_error(train_pred, training_target)
print('Train RMSE is %.2f' % sqrt(train_mse))

Train RMSE is 0.11


In [ ]:
test_pred = exported_pipeline.predict(testing_features)
test_mse = mean_squared_error(test_pred, testing_target)
print('Test RMSE is %.2f' % sqrt(test_mse))

Test RMSE is 0.19


In [ ]:
df_range = pd.DataFrame(results)
df_range

,0
0,-1.000000
1,1.000000
2,2.000000
3,1.000000
4,1.000000
...,...
10720,-1.000000
10721,-0.714286
10722,1.000000
10723,1.000000


In [ ]:
res = exported_pipeline.predict([[92269179, 12]])
pd.DataFrame(res)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,0
0,1.0


In [ ]:
from datetime import datetime, timedelta

def calculate_outbound_date(product_id, qty, inbound_date):
  res = int(exported_pipeline.predict([[product_id, qty]])[0])
  date = datetime.strptime(inbound_date, "%Y-%m-%d")
  outbound_date = date + timedelta(days=res)
  return outbound_date

print('Date outbound:', calculate_outbound_date(92269179, 15, "2022-05-11"))

Date outbound: 2022-05-12 00:00:00


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pickle

rfr = "rfr.pkl"

# save model
pickle.dump(exported_pipeline, open(rfr, "wb"))

# load model
loaded_model = pickle.load(open(rfr, "rb"))

In [ ]:
loaded_model.predict([[92269179, 12]])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1.])

In [ ]:
# df.to_csv(r'dataset.csv', index=False)